In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [104]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.prompts.chat import ChatPromptTemplate
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.messages import get_buffer_string
from langchain.schema import format_document

In [91]:
ak = "api-key"

In [66]:
url = "https://www.fmkorea.com/search.php?act=IS&is_keyword=kt+%EC%9D%B8%ED%84%B0%EB%84%B7&mid=home&where=comment&page=3"
response = requests.get(url)

In [67]:
dom=BeautifulSoup(response.text,"html.parser")

In [70]:
dom.select(".time")[2].text

'2023-12-20 10:03'

In [16]:
elements = dom.select("dt>a")

In [17]:
elements

[<a href="/6524513041/6524644886#comment_6524644886">[유머/움짤/이슈] 트위치 CEO가 거짓말한 거였음<br/>병신들보면 방사능어쩌구선동하던사람들 보는거같음 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 진짜 존나모자라보임 국제정세때문에 발빼는거지 한국에서 망사용내면 지금 빅테크기업들 탈세존나해서 여러나라에서 해외기업 망사용료존나때릴준비하니깐 선례안남길려고 발빼는거지 실제로 우리나라에서 탈세해서 얼마나 해쳐먹었을지생각하면 걍 꼴이좋아 ㅋㅋ 아마존프라임등등 좆지랄해봤는게 쿠팡에밀려서 모...</a>,
 <a href="/6524513041/6524643328#comment_6524643328">[유머/움짤/이슈] 트위치 CEO가 거짓말한 거였음<br/>https://www.sharedit.co.kr/posts/7290 이제 한국의 수치만 자세히 살펴볼까요? 왼쪽이 유선, 오른쪽이 무선 <b class="searchContextDoc">인터넷</b> 트래픽 입니다. 유선과 무선 모두 월별 트래픽 양은 2022년에 늘어나지만 유선은 68%, 무선은 188%로 그 증가폭에서 차이가 3배 가까이 납니다. 전체 트래픽에서 유선이 차지하는 비중은 2017년 42%에서 2022년 28%로 14%나 줄어드는 반면 무선은 2017년 51%...</a>,
 <a href="/6524637758/6524641536#comment_6524641536">[PC/콘솔 게임 ] M1칩<br/>클라우드는 <b class="searchContextDoc">인터넷</b>만 좋으면 돌아감</a>,
 <a href="/6524545297/6524639258#comment_6524639258">[주식] 핫딜 시모에 대항한 펨코의 봄 ㅋㅋㅋㅋㅋㅋ<br/>애드 가드 설치완료 ... 삼성페이만 통과하게 해둠... 삼성<b class="searchContextDoc">인터넷</b> 차단기능은 효과가 없네요 ㅜ</a>,
 <a href="/6524528853/6524638783#comme

In [29]:
str(elements[0]).split("<br/>")[1][:-4]

'병신들보면 방사능어쩌구선동하던사람들 보는거같음 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 진짜 존나모자라보임 국제정세때문에 발빼는거지 한국에서 망사용내면 지금 빅테크기업들 탈세존나해서 여러나라에서 해외기업 망사용료존나때릴준비하니깐 선례안남길려고 발빼는거지 실제로 우리나라에서 탈세해서 얼마나 해쳐먹었을지생각하면 걍 꼴이좋아 ㅋㅋ 아마존프라임등등 좆지랄해봤는게 쿠팡에밀려서 모...'

In [31]:
str(elements[0]).split("<br/>")[0].split('] ')[1]

'트위치 CEO가 거짓말한 거였음'

In [48]:
datetime.datetime.strptime(dom.select_one(".time").text,'%Y-%m-%d %H:%M')<datetime.datetime(2023, 12, 21, 9, 41)

True

In [86]:
d = {'comment':[],'title':[],'date':[]}
for i in range(1,101):
    url = f"https://www.fmkorea.com/search.php?act=IS&is_keyword=kt+인터넷&mid=home&where=comment&page={i}"
    response = requests.get(url)
    dom=BeautifulSoup(response.text,"html.parser")
#     if datetime.datetime.strptime(dom.select_one(".time").text,'%Y-%m-%d %H:%M') < datetime.datetime(2023, 11, 20, 9, 41):
#         break
    elements = dom.select("dt>a")
    dates = dom.select(".time")
    for j in range(len(elements)):
        title  = str(elements[j]).split("<br/>")[0].split('] ')[1]
        comment = str(elements[j]).split("<br/>")[1][:-4].replace('<b class="searchContextDoc">','').replace('</b>','')
        date = dates[j].text
        d['title'].append(title)
        d['comment'].append(comment)
        d['date'].append(date)

df = pd.DataFrame(d)
df

,comment,title,date
0,예전부터 인터넷문제잇던데 왜안바꾸지 건물이 kt밖에못쓰나??,만식이방 억까지리네,2023-12-19 20:39
1,그리고 인터넷 kt는 쓰레기니 비추,월세사는데 인터넷 따로 설치해보신분 있음?,2023-12-18 21:53
2,인터넷은 무조건 KT 라고 생각함,인터넷 kt에서 옮기려는데,2023-12-18 18:21
3,5.21. 인터넷 방송인 성훈에게 유튜브 채널 테러 및 비난5.22. 2022년 클...,현재 케리아 초대석 예고에서 말실수 했다고 논란중인 침착맨,2023-12-18 14:21
4,Kt개잡놈들은 이럴때도 도움이안됨.. 인터넷도 개같이 느리고,다들 유튜브뮤직 계속 쓰실건가요??,2023-12-18 10:01
...,...,...,...
995,같은아파트내에서 분가했는데 본가 - skt / 신혼집 - kt 품질은 skt가 더 ...,구축 아파트 인터넷 선택 질문드립니다,2023-06-29 15:56
996,지금 kt에 다묶여있는데 인터넷 티비는 3사꺼쓰고 자급제+알뜰폰 가면 그게 더 쌈?...,데이터팔이새끼들 다 뒤졌으면,2023-06-29 14:08
997,인터넷은 KT,kt랑 lg랑 인터넷 어디가 더 좋으려나,2023-06-29 08:44
998,"압도적 인터넷 속도, 다양한 혜택, 스포츠의 지배자 “KT”",힘쎄고 강한 오후,2023-06-28 17:53


In [88]:
df.head(30)

,comment,title,date
0,예전부터 인터넷문제잇던데 왜안바꾸지 건물이 kt밖에못쓰나??,만식이방 억까지리네,2023-12-19 20:39
1,그리고 인터넷 kt는 쓰레기니 비추,월세사는데 인터넷 따로 설치해보신분 있음?,2023-12-18 21:53
2,인터넷은 무조건 KT 라고 생각함,인터넷 kt에서 옮기려는데,2023-12-18 18:21
3,5.21. 인터넷 방송인 성훈에게 유튜브 채널 테러 및 비난5.22. 2022년 클...,현재 케리아 초대석 예고에서 말실수 했다고 논란중인 침착맨,2023-12-18 14:21
4,Kt개잡놈들은 이럴때도 도움이안됨.. 인터넷도 개같이 느리고,다들 유튜브뮤직 계속 쓰실건가요??,2023-12-18 10:01
5,ㅁ?ㄹ kt 인터넷 쓰다가 데였나?,그 호감고닉 능지 딸려서 계속 걸리는게 웃김,2023-12-18 00:24
6,아니면 반대로 인터넷선을 본인방으로 연장하시고 거기 공유기두시고 거실에 증폭기 두는...,형들 여기다가 pcie 랜카드 꽂을수 있을까?,2023-12-17 03:49
7,"나도 알뜰폰인데 집에 케이블 티비,인터넷이 KT 그래서 쌓인 포인트로 매달 저거 이용",[KT 멤버십,2023-12-16 14:09
8,kt마다 좀 다른가 나도 kt이긴 한데 거의 단 한번도 인터넷 문제 있었던 적이 없...,Kt 인터넷 ㅈㄴ 빡친다,2023-12-15 21:54
9,월세인데 건물 인터넷 kt 계약임,Kt 인터넷 ㅈㄴ 빡친다,2023-12-15 21:52


In [92]:
loader = DataFrameLoader(df,page_content_column='comment')
docs = loader.load()
vectorstore = FAISS.from_documents(docs,embedding=OpenAIEmbeddings(openai_api_key=ak))

In [95]:
retriever = vectorstore.as_retriever()
bm25_retriever = BM25Retriever.from_documents(docs)

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

llm = ChatOpenAI(openai_api_key=ak, temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=ensemble_retriever, llm=llm
)

In [98]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


template = """Guess the answer in korean about the question by referring the following context,:{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)


DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [105]:
# MultiQueryRetriever 이용
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(openai_api_key=ak,temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever_from_llm,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(openai_api_key=ak),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [110]:
inputs = {"question": "KT 인터넷과 SKT 인터넷 어떤게 좋아?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='KT 인터넷이 좋다.'),
 'docs': [Document(page_content='모바일 SKT &gt; KT &gt;&gt;&gt;&gt;&gt;&gt; LG 인터넷 KT &gt; SKT &gt;&gt;&gt;&gt;&gt;&gt; LG', metadata={'title': '유플 쓰다가 SKT로 갈아탔는데', 'date': '2023-10-22 00:13'}),
  Document(page_content='인터넷은 kt쓰라던데 skt가 나음?', metadata={'title': '현재 ㅈ됐다는 KT 통신사 근황', 'date': '2023-11-08 12:26'}),
  Document(page_content='인터넷은 kt고 폰은 skt인데 되려나요', metadata={'title': '플스 KT쓰시는분들 현질하실때 이벤트 참여하세요', 'date': '2023-10-13 13:23'}),
  Document(page_content='인터넷의 가장큰문제는 kt가아니면 폰이나 인터넷둘중하난 쓰레길써야한다는거', metadata={'title': '통신사 불편한진실 간단히 쓴 인터넷편.. 팁 공유해', 'date': '2023-08-07 09:47'}),
  Document(page_content='인터넷은 KT 모바일은 SKT 아니었나', metadata={'title': '근데 진짜 왜 kt 쓰는거?', 'date': '2023-08-12 20:52'}),
  Document(page_content='인터넷은 kt 모바일은 skt', metadata={'title': 'Kt 모바일이 ㄹㅇ 개좆이었구나', 'date': '2023-09-28 16:44'}),
  Document(page_content='인터넷 KT --&gt; SKT 변경? ㅋㅋ', metadata={'title': '봉준이 구국의 결단 예상', 'date': '2023-08-05 19:06'}),
  Document(page_c